In [9]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Dropout,Flatten,MaxPooling2D,BatchNormalization

model=Sequential()

model.add(Conv2D(128,(3,3),input_shape=(100,100,1),activation='relu',kernel_initializer='orthogonal'))
model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3),activation='relu',kernel_initializer='orthogonal'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3),activation='relu',kernel_initializer='orthogonal'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 98, 98, 128)       1280      
_________________________________________________________________
batch_normalization_9 (Batch (None, 98, 98, 128)       512       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 96, 96, 64)        73792     
_________________________________________________________________
batch_normalization_10 (Batc (None, 96, 96, 64)        256       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 46, 46, 32)        18464     
_________________________________________________________________
batch_normalization_11 (Batc (None, 46, 46, 32)       

In [10]:
model.load_weights('model-010.h5')

# Web Cam

In [11]:
import cv2 as cv
import random
import numpy as np

source = cv.VideoCapture(0)

modelFile = "face_model/res10_300x300_ssd_iter_140000.caffemodel"
configFile = "face_model/deploy.prototxt.txt"
net = cv.dnn.readNetFromCaffe(configFile, modelFile)

width=300
height=300
mean = [104,117,123]
confidence_level = 0.7

img_size=100

while cv.waitKey(1) != 27:
    has_video, frame=source.read()
    
    if not has_video:
        break
    frame = cv.flip(frame,1)
    frame_height = frame.shape[0]
    frame_width = frame.shape[1]
    
    blob =cv.dnn.blobFromImage(frame,1.0,(width,height), mean,swapRB =False,crop =False)
    net.setInput(blob)
    detections = net.forward()
    
    try:
    
        for detection in range(detections.shape[2]):
            confidence = detections[0,0,detection,2]
            if confidence > confidence_level:
                x_left_bottom = int(detections[0,0,detection,3] *frame_width)
                y_left_bottom = int(detections[0,0,detection,4] *frame_height)
                x_right_top = int(detections[0,0,detection,5] *frame_width)
                y_right_top = int(detections[0,0,detection,6] *frame_height)

                img = frame[y_left_bottom:y_right_top,x_left_bottom:x_right_top]

                img = cv.resize(img,(100,100))
                img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
                img = img/255
                img = img.reshape(1,100,100,1)

                result = model.predict(img)
                label = np.argmax(result,axis=1)[0]
                accuracy=float(np.max(result,axis=1)[0]*100)

                category_dict={0:'WEARING A MASK',1:'NOT WERAING A MASK'}
                category =  category_dict[label]

                if accuracy>75 and category == "WEARING A MASK":
                    color = (0,255,0)
                    display_label="WEARING A MASK"
                elif category == "NOT WERAING A MASK":
                    color = (0,0,255)
                    display_label="NOT WERAING A MASK"


                cv.rectangle(frame,(x_left_bottom,y_left_bottom),(x_right_top,y_right_top),color)
                acc_label=f"{category}: {accuracy:.4f}"
                label_size,base_line = cv.getTextSize(acc_label,cv.FONT_HERSHEY_SIMPLEX,0.5,1)
                cv.rectangle(frame ,(x_left_bottom,y_left_bottom-label_size[1]),
                             (x_left_bottom+label_size[0],y_left_bottom+base_line),color,cv.FILLED)
                cv.putText(frame,acc_label,(x_left_bottom,y_left_bottom),cv.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255))

                cv.imshow("Camera_Preview",frame)
            else:
                cv.imshow("Camera_Preview",frame)
    except Exception as e:
            print(e)
            cv.imshow("Camera_Preview",frame)
source.release()
cv.destroyAllWindows()